# УЧЕСТЬ НАЗВАНИЯ И ТИПЫ ДАННЫХ В ЕМИАС АЙДИ

# ПРОВЕРИТЬ НА ВЫХОДЕ ПОСЛЕ МЕРДЖА 

# ДОБАВИТЬ НЕСКОЛЬКО ПРОВЕРОК ПРИ ФОРМИРОВАНИИ ЗАДАНИЙ, ЧТОБЫ ТОЧНО ПОПАЛО, ЧТО ДОЛЖНО ПОПАСТЬ И ТОЧНО НЕ ПОПАЛО, ЧТО НЕ ДОЛЖНО ПОПАСТЬ

# ПРОПИСАТЬ ВОЗМОЖНОСТЬ ИСКЛЮЧЕНИЯ И ОТМЕНЫ ИСКЛЮЧЕНИЯ ПАЦИЕНТОВ

# ДОБАВИТЬ ПОСЛЕ ЧТЕНИЕ КОПИРАНИЕ ФАЙЛА, ЧТОБЫ ПРИ КАКИХ-ЛИБО ПРАВКАХ НЕ ПРИХОДИЛОСЬ ПОВТОРНО ЖДАТЬ СЧИТЫВАНИЯ ФАЙЛА

# ПРОПИСАТЬ ПЕРЕИМЕНОВАНИЕ ВСЕХ СТОЛБЦОВ ЕМИАС В ОДИН ОБЩИЙ ШАБЛОН, А ТАКЖЕ ТАМ ЖЕ ПРИВЕСТИ ТИП ДАННЫХ ЕМИАС К ОДНОМУ ТИПУ

# СДЕЛАТЬ ТАК, ЧТОБЫ ДАННЫЕ ПО ЗВОНКАМ ОТНОСИЛИСЬ ТОЛЬКО К ЭТОМУ МЕСЯЦУ

# ОБЯЗАТЕЛЬНО ИСКЛЮЧИТЬ ТЕСТОВЫЕ И ВХОДЯЩИЕ

# СДЕЛАТЬ ТАК ЧТОБЫ МО БЫЛА ВСЕГДА ПОСЛЕДНЯЯ, ЕСЛИ МО НОВАЯ НЕ ВХОДИТ ВЫВОДИТЬ СТАТУС СМЕНА МО
# ТАКЖЕ ПРОВЕРЯТЬ ЕСТЬ ЛИ ВРАЧ ИЗ НАШЕЙ МО В НАШЕМ РАСКРЕПЛЕНИИ, ЕСЛИ НЕТ ТО КИДАТЬ ЕГО РАНДОМНОМУ ПВ ИЗ ЭТОЙ МО

Пациент может сменить врача, 2 августа был один врач мы закрепили его, потом совсем другой, новый врач которого нет в закреплении, 15 августа обновим, а до этого врач которого нет в закреплении падает случайному помощнику из этой мо

- какие файлы нам нужно подгружать?
    - на первичном этапе:
        * выгрузка по пациентам к этому числу
        * выгрузка по звонкам к этому числу
        * список "все прошли" к этому числу
    - в текущей работе:
        * сегодняшняя выгрузка по пациентам
        * сегодняшняя выгрузка по звонкам к этому числу
        * сегодняшний список "все прошли" к этому числу
        * раскрепление по врачам и помощникам - для равномерного распределения списка по дням по помощникам

In [1]:
# на случай необходимости импортирования чего либо из pdn
import sys
path_to_pdn = r"C:\Users\murad_satabaev\Documents\GitHub\pdn_"
# сюда внутри кавычек вставляем путь, где ледит репозиторий пдн
sys.path.insert(0, path_to_pdn)
from pdn.download_data.download_calls_data.download_calls_data import download

['calls_reports']


In [2]:
import os
import pandas as pd
import datetime as dt
import numpy as np

In [3]:
# путь папки с ЕМИАС данными
emias_data_path = r'\\t999\Сетевой диск\pdn\data\emias_data\xlsx'
emias_data_path = emias_data_path.split('\\')
emias_data_path = os.path.join(r'\\', *emias_data_path)

In [4]:
# путь папки с данными по звонкам
calls_data_path = r'\\t999\Сетевой диск\pdn\data\from_call_program'
calls_data_path = calls_data_path.split('\\')
calls_data_path = os.path.join(r'\\', *calls_data_path)

In [5]:
# путь папки с данными по пациентам, которые говорят, что все прошли
all_done_patients_data_path = r''
all_done_patients_data_path = all_done_patients_data_path.split('\\')
all_done_patients_data_path = os.path.join(r'\\', *all_done_patients_data_path)

In [6]:
# путь папки с данными по распределению помощников врачей по врачам
assistant_to_doctor_distribution_path = r''
assistant_to_doctor_distribution_path = assistant_to_doctor_distribution_path.split('\\')
assistant_to_doctor_distribution_path = os.path.join(r'\\', *assistant_to_doctor_distribution_path)

In [7]:
# путь файла с текущими статусами пациентов
patient_current_statuses_path = r''
patient_current_statuses_path = patient_current_statuses_path.split('\\')
patient_current_statuses_path = os.path.join(r'\\', *patient_current_statuses_path)

In [8]:
# путь папки для кратковременных несистемных исключений
temporary_exception_path = r''
temporary_exception_path = temporary_exception_path.split('\\')
temporary_exception_path = os.path.join(r'\\', *temporary_exception_path)

In [20]:
# cловарь "номер_dat": "месяц", в который мы добавляем список месяцев, которые сейчас в работе

dat_month = dict()

# dat_month['may'] = '3196'
# dat_month['june'] = '3437'
# dat_month['july'] = '3582'
# dat_month['august'] = '3833'
dat_month['august_'] = '3940'

In [21]:
today = pd.Timestamp(dt.datetime.today().date())

today_ymd = today.strftime('%Y%m%d')
today_y_m_d = today.strftime('%Y_%m_%d')

yesterday = today - dt.timedelta(days=1)

yesterday_ymd = yesterday.strftime('%Y%m%d')
yesterday_y_m_d = yesterday.strftime('%Y-%m-%d')

In [22]:
# формируем даты, которые являются рабочими днями циклов

stages = dict()

stages[1] = pd.date_range('20220802', '20220810')
stages[2] = pd.date_range('20220811', '20220819')
stages[3] = pd.date_range('20220820', '20220831')

stages_and_days = dict()

for stage, dates in stages.items():
    
    stage_day_number = 0
    
    for weekday in [x.date() for x in dates if not x.day_name() in {'Saturday', 'Sunday'} ]:
        stages_and_days[weekday] = dict()
        stages_and_days[weekday]['stage'] = stage
        stage_day_number += 1
        stages_and_days[weekday]['stage_day'] = stage_day_number\
        
stages_df = pd.DataFrame(stages_and_days).T.reset_index()

In [23]:
column_names = {'Emiasid': 'emiasid', 'Emiasid ': 'emiasid' , 'emiasid ': 'emiasid', 'EMIAS_ID': 'emiasid', 'МО': 'mo', 
                'МУ': 'mu', 'Логин оператора': 'login', 'Дата и время звонка': 'call_date', 'Результат звонка': 'call_result',
                'Результат звонка ': 'call_result', 'Разговор состоялся': 'talk_result', 'Комменатрий': 'comment', 
                'Дата и время переноса': 'next_call_date', 'Контактный телефон': 'phone_number', 'ФИО врача': 'doctor_fio', 
                'ФИО помощника врача': 'assistant_fio', 'Логин помощника врача': 'login', 'Зал №': 'hall_number',
                'Логин naumen': 'login', 'МО закрепления': 'mo', 'Отсутствует на рабочем месте': 'absence'}

In [24]:
rename_mo = {'ГБУЗ Г МОСКВЫ ГП № 170 ДЗМ': 'ГП 170', ' ГП 6 ДЗМ': 'ГП 6', 'ГБУЗ КДЦ 6 ДЗМ':'КДЦ 6', 
             'ГБУЗ "ГП № 134 ДЗМ"': 'ГП 134', 'КДЦ 4': 'КДЦ 4', 'ГБУЗ ДКЦ 1 ДЗМ': 'ДКЦ 1', 'ГБУЗ ГП 23 ДЗМ': 'ГП 23',
             'ГБУЗ ГП 109 ДЗМ': 'ГП 109', 'ГБУЗ ГП 11 ДЗМ': 'ГП 11', 'ГБУЗ ГП 64 ДЗМ': 'ГП 64', 'ГБУЗ ГП 191 ДЗМ': 'ГП 191', 
             'ГБУЗ ГП 220 ДЗМ': 'ГП 220', 'ГБУЗ "ГП № 218 ДЗМ"': 'ГП 218',  'ГБУЗ ГП 107 ДЗМ': 'ГП 107', 
             'ГБУЗ "ГП № 22 ДЗМ"': 'ГП 22', 'ГБУЗ ДЦ 5 ДЗМ': 'ДЦ 5', 'ГБУЗ ГП 210 ДЗМ': 'ГП 210', 'ГБУЗ ГП 214 ДЗМ': 'ГП 214', 
             'ГБУЗ ГП 2 ДЗМ': 'ГП 2', 'ГБУЗ КДП 121 ДЗМ': 'КДП 121', 'ГБУЗ ГП 66 ДЗМ': 'ГП 66',  'ГБУЗ ДЦ 3 ДЗМ ': 'ДЦ 3',
             'ГБУЗ ГП 36 ДЗМ': 'ГП 36',  'ГБУЗ ГП 62 ДЗМ ': 'ГП 62'}

In [25]:
current_mo = set(rename_mo.values())
current_mo.add(np.NaN)

In [26]:
%%time
# Блок,который собирает к данному числу данные из ЕМИАС по emiasid и статусам "Умер", "Госпитализирован" и "Смена МО"
# тут ничего менять не нужно, нужно изменить словарь dat_month и добавить туда новые вводные либо отключить старые

patient_current_statuses_path = os.getcwd()

wrong_numbers = {'0000000000', '1111111111', '3262916916', '1234567899', '1236547890', '3690550657',
'0007618491', '1659656596', '9999999999', '3333333333', '7777777777'}

for month, dat_number in dat_month.items():
    
    distinct_month_patients = pd.DataFrame(columns=['emiasid', 'date', 'status', 'old_comment', 'next_call_date'])
    distinct_month_patients_current_statuses_path = os.path.join(patient_current_statuses_path, 
                                                                 rf'_{month}_patients_current_statuses_{today_y_m_d}.xlsx')

    
    for patients_emias_data_file in os.listdir(emias_data_path):
               
        
        if dat_number in patients_emias_data_file and '.xlsx' in patients_emias_data_file and \
        not '~$' in patients_emias_data_file:
            
            
            patients_emias_data = pd.read_excel(os.path.join(emias_data_path, patients_emias_data_file), 
                                                usecols=['mo', 'emiasid', 'phone', 'doctor_fio', 'diagnoz_hospital', 'Умер'])
            
            patients_emias_data['mo'] = patients_emias_data['mo'].apply(lambda x: rename_mo[x] if x in set(rename_mo.keys())
                                                                       else x)
            print('done')
            print()
            
            patients_emias_data = patients_emias_data.rename(columns={'Умер': 'died'})
            patients_emias_data['emiasid'] = patients_emias_data['emiasid'].astype(str).str.replace('\.0', '')
            patients_emias_data = patients_emias_data.sort_values(by='emiasid').reset_index(drop=True)
            
            
            if patients_emias_data[~patients_emias_data.emiasid.isin(distinct_month_patients.emiasid)].shape[0] != 0:
                
                distinct_month_patients = pd.concat([distinct_month_patients, patients_emias_data[~patients_emias_data \
                .emiasid.isin(distinct_month_patients.emiasid)][['emiasid', 'doctor_fio', 'mo']]])
                
                distinct_month_patients = distinct_month_patients.sort_values(by='emiasid').reset_index(drop=True)

                distinct_month_patients['date'] = distinct_month_patients['date'].fillna(dt.date.fromtimestamp(os.path \
                .getctime(os.path.join(emias_data_path,patients_emias_data_file))))
                
                distinct_month_patients['status'] = distinct_month_patients['status'].fillna('без статуса')
                   

            died_patients = set(patients_emias_data[patients_emias_data.died == 'да'].emiasid)
            died_patients.add('22748553')
            died_patients.add('18525757')
            died_patients.add('17279632')
            died_patients.add('219042416')
            died_patients.add('21039599')
            died_patients.add('808987316')
            died_patients.add('23903336')
            
            wrong_number_patients = set(patients_emias_data[patients_emias_data.phone.isin(wrong_numbers)].emiasid)
            
            hospitalized_patients = set(patients_emias_data[~patients_emias_data.diagnoz_hospital.isna()].emiasid)
            
            condition_for_died_patients = (distinct_month_patients.emiasid.isin(died_patients) &
            (distinct_month_patients.status != 'Умер'))
            distinct_month_patients.loc[condition_for_died_patients, 'date'] = dt.date.fromtimestamp(
                    os.path.getctime(os.path.join(emias_data_path,patients_emias_data_file)))
            distinct_month_patients.loc[condition_for_died_patients, 'status'] = 'Умер'
            

            condition_for_hospitalization = (distinct_month_patients.emiasid.isin(hospitalized_patients) &
            (distinct_month_patients.status != 'Умер'))
            distinct_month_patients.loc[condition_for_hospitalization, 'date'] = dt.date.fromtimestamp(
                    os.path.getctime(os.path.join(emias_data_path,patients_emias_data_file)))
            distinct_month_patients.loc[condition_for_hospitalization, 'status'] = 'Госпитализирован'
            
            
            common_patients_emias_data = set(patients_emias_data.emiasid) & set(distinct_month_patients.emiasid)


            distinct_month_patients = distinct_month_patients.rename(columns={'doctor_fio': 'old_doctor_fio', 'mo':'old_mo'}) \
            .merge(patients_emias_data[['emiasid', 'doctor_fio', 'mo']] \
            .rename(columns={'doctor_fio': 'new_doctor_fio', 'mo':'new_mo'}))
            
            distinct_month_patients.loc[~distinct_month_patients['new_doctor_fio'].isna(), 'doctor_fio'] = \
            distinct_month_patients.loc[~distinct_month_patients['new_doctor_fio'].isna(), 'new_doctor_fio']
            distinct_month_patients.loc[distinct_month_patients['new_doctor_fio'].isna(), 'doctor_fio'] = \
            distinct_month_patients.loc[distinct_month_patients['new_doctor_fio'].isna(), 'old_doctor_fio']
            
            distinct_month_patients.loc[~distinct_month_patients['new_mo'].isna(), 'mo'] = \
            distinct_month_patients.loc[~distinct_month_patients['new_mo'].isna(), 'new_mo']
            distinct_month_patients.loc[distinct_month_patients['new_mo'].isna(), 'mo'] = \
            distinct_month_patients.loc[distinct_month_patients['new_mo'].isna(), 'old_mo']
            
            
            distinct_month_patients = distinct_month_patients.drop(columns=['old_doctor_fio', 'new_doctor_fio', 'old_mo',
                                                                           'new_mo'])
            
            
            
            distinct_month_patients.loc[~distinct_month_patients.mo.isin(current_mo) & 
                                distinct_month_patients.emiasid.isin(common_patients_emias_data), 'date'] = \
            dt.date.fromtimestamp(os.path.getctime(os.path.join(emias_data_path,patients_emias_data_file)))
            
            distinct_month_patients.loc[~distinct_month_patients.mo.isin(current_mo) & 
                                distinct_month_patients.emiasid.isin(common_patients_emias_data), 'doctor_fio'] = 'Смена МО'
            distinct_month_patients.loc[~distinct_month_patients.mo.isin(current_mo) & 
                                distinct_month_patients.emiasid.isin(common_patients_emias_data), 'status'] = 'Смена МО'
            
            
            
            condition_for_wrong_number = (distinct_month_patients.emiasid.isin(wrong_number_patients) &
            (~distinct_month_patients.status.isin({'Умер', 'Госпитализирован', 'Смена МО'})))        
            distinct_month_patients.loc[condition_for_wrong_number, 'status'] = 'Некорректный номер'
            
            

            print(patients_emias_data_file, ' обработан')
            
    distinct_month_patients.to_excel(distinct_month_patients_current_statuses_path, index=False)
    
    print('этот месяц окончен')
    print()

print('Конец')

done

DAT_3940_выгрузка_списков_на_обзвон_помощниками_2022-08-16.xlsx  обработан


<timed exec>:32: FutureWarning: The default value of regex will change from True to False in a future version.


этот месяц окончен

Конец
CPU times: total: 2.5 s
Wall time: 2.59 s


In [ ]:
## 10min 3s

In [ ]:
# %%time
# # Блок,который собирает сегодняшние данные из ЕМИАС по emiasid и статусам "Умер", "Госпитализирован" и "Смена МО"

# patient_current_statuses_path = os.getcwd()

# for month, dat_number in dat_month.items():
    
# #     distinct_month_patients = pd.DataFrame(columns=['emiasid', 'date', 'status', 'old_comment', 'next_call_date'])
#     distinct_month_patients_last_statuses_path = os.path.join(patient_current_statuses_path, 
#                                                                  rf'{month}_patients_current_statuses_{yesterday_y_m_d}.xlsx')
# #     надо будет изменить путь, откуда берется предыдущие статусы по месяцу
    
#     distinct_month_patients = pd.read_excel(distinct_month_patients_last_statuses_path)
#     distinct_month_patients['emiasid'] = distinct_month_patients['emiasid'].astype(str).str.replace('\.0', '')
#     distinct_month_patients['date'] = distinct_month_patients['date'].apply(lambda x: pd.to_datetime(x).date())
    
#     distinct_month_patients_current_statuses_path = os.path.join(patient_current_statuses_path, 
#                                                                  rf'{month}_patients_current_statuses_{today_y_m_d}.xlsx')
#     distinct_month_patients_full_info = pd.DataFrame(columns=['emiasid'])

#     patients_emias_data_file = [x for x in os.listdir(emias_data_path) 
#                                 if dat_number in x and '.xlsx' in x and not '~$' in x][-1]
    
    
#     patients_emias_data = pd.read_excel(os.path.join(emias_data_path, patients_emias_data_file), 
#                                         usecols=['mo', 'emiasid', 'doctor_fio', 'diagnoz_hospital', 'Умер'])

#     patients_emias_data['mo'] = patients_emias_data['mo'].apply(lambda x: rename_mo[x] if x in set(rename_mo.keys())
#                                                                else x)
#     print('done')
#     print()

#     patients_emias_data = patients_emias_data.rename(columns={'Умер': 'died'})
#     patients_emias_data['emiasid'] = patients_emias_data['emiasid'].astype(str).str.replace('\.0', '')
#     patients_emias_data = patients_emias_data.sort_values(by='emiasid').reset_index(drop=True)


#     if patients_emias_data[~patients_emias_data.emiasid.isin(distinct_month_patients.emiasid)].shape[0] != 0:

#         distinct_month_patients = pd.concat([distinct_month_patients, patients_emias_data[~patients_emias_data \
#         .emiasid.isin(distinct_month_patients.emiasid)][['emiasid', 'doctor_fio', 'mo']]])

#         distinct_month_patients = distinct_month_patients.sort_values(by='emiasid').reset_index(drop=True)

#         distinct_month_patients['date'] = distinct_month_patients['date'].fillna(dt.date.fromtimestamp(os.path \
#         .getctime(os.path.join(emias_data_path,patients_emias_data_file))))

#         distinct_month_patients['status'] = distinct_month_patients['status'].fillna('без статуса')


#     died_patients = set(patients_emias_data[patients_emias_data.died == 'да'].emiasid)
#     hospitalized_patients = set(patients_emias_data[~patients_emias_data.diagnoz_hospital.isna()].emiasid)

#     condition_for_died_patients = (distinct_month_patients.emiasid.isin(died_patients) &
#     (distinct_month_patients.status != 'Умер'))
#     distinct_month_patients.loc[condition_for_died_patients, 'date'] = dt.date.fromtimestamp(
#             os.path.getctime(os.path.join(emias_data_path,patients_emias_data_file)))
#     distinct_month_patients.loc[condition_for_died_patients, 'status'] = 'Умер'

#     condition_for_hospitalization = (distinct_month_patients.emiasid.isin(hospitalized_patients) &
#     (distinct_month_patients.status != 'Умер'))
#     distinct_month_patients.loc[condition_for_hospitalization, 'date'] = dt.date.fromtimestamp(
#             os.path.getctime(os.path.join(emias_data_path,patients_emias_data_file)))
#     distinct_month_patients.loc[condition_for_hospitalization, 'status'] = 'Госпитализирован'


#     common_patients_emias_data = set(patients_emias_data.emiasid) & set(distinct_month_patients.emiasid)


#     distinct_month_patients = distinct_month_patients.rename(columns={'doctor_fio': 'old_doctor_fio', 'mo':'old_mo'}) \
#     .merge(patients_emias_data[['emiasid', 'doctor_fio', 'mo']] \
#     .rename(columns={'doctor_fio': 'new_doctor_fio', 'mo':'new_mo'}))

#     distinct_month_patients.loc[~distinct_month_patients['new_doctor_fio'].isna(), 'doctor_fio'] = \
#     distinct_month_patients.loc[~distinct_month_patients['new_doctor_fio'].isna(), 'new_doctor_fio']
#     distinct_month_patients.loc[distinct_month_patients['new_doctor_fio'].isna(), 'doctor_fio'] = \
#     distinct_month_patients.loc[distinct_month_patients['new_doctor_fio'].isna(), 'old_doctor_fio']

#     distinct_month_patients.loc[~distinct_month_patients['new_mo'].isna(), 'mo'] = \
#     distinct_month_patients.loc[~distinct_month_patients['new_mo'].isna(), 'new_mo']
#     distinct_month_patients.loc[distinct_month_patients['new_mo'].isna(), 'mo'] = \
#     distinct_month_patients.loc[distinct_month_patients['new_mo'].isna(), 'old_mo']


#     distinct_month_patients = distinct_month_patients.drop(columns=['old_doctor_fio', 'new_doctor_fio', 'old_mo',
#                                                                    'new_mo'])



#     distinct_month_patients.loc[~distinct_month_patients.mo.isin(current_mo) & 
#                         distinct_month_patients.emiasid.isin(common_patients_emias_data), 'date'] = \
#     dt.date.fromtimestamp(os.path.getctime(os.path.join(emias_data_path,patients_emias_data_file)))

#     distinct_month_patients.loc[~distinct_month_patients.mo.isin(current_mo) & 
#                         distinct_month_patients.emiasid.isin(common_patients_emias_data), 'doctor_fio'] = 'Смена МО'
#     distinct_month_patients.loc[~distinct_month_patients.mo.isin(current_mo) & 
#                         distinct_month_patients.emiasid.isin(common_patients_emias_data), 'status'] = 'Смена МО'

#     print(patients_emias_data_file, ' обработан')

#     distinct_month_patients.to_excel(os.path.join(patient_current_statuses_path, 
#                                                     rf'{month}_patients_current_statuses_{today_y_m_d}.xlsx'), index=False)

#     print('этот месяц окончен')
#     print()

# print('Конец')
    

# На этом заканчивается формирование статусов из емиас_даты

# Теперь начинаем формировать статусы из данных по звонкам

# НУЖНО ИЗМЕНИТЬ ПОДГРУЗКУ CALLS DATA, ПОСТАВИТЬ ИЗ FROM CALL PROGRAM

In [27]:
print('Конец')

Конец


In [28]:
assistant_to_doctor_distribution = pd.read_excel('Основное Раскрепление Август.xlsx')

In [29]:
assistant_to_doctor_distribution = assistant_to_doctor_distribution.rename(columns=column_names)

17332960 - Перенос звонка на 19.08

In [30]:
calls_data_path = rf'calls_data_20220802_20220815.xlsx'

In [31]:
%%time
# Блок,который собирает данные из звонилки по emiasid и формирует статусы

calls_data_reading = pd.read_excel(calls_data_path)
calls_data = calls_data_reading.copy()


calls_data = calls_data.rename(columns=column_names)
calls_data = calls_data[['mo', 'mu', 'emiasid', 'phone_number', 'login', 'call_date', 'call_result', 'talk_result', 
                         'next_call_date', 'comment']]

calls_data['emiasid'] = calls_data['emiasid'].astype(str).str.replace('\.0', '')
calls_data['call_date'] = calls_data['call_date'].apply(lambda x: pd.to_datetime(x, dayfirst=True).date())
calls_data['next_call_date'] = calls_data['next_call_date'].apply(lambda x: pd.to_datetime(x, dayfirst=True).date())



for month, dat_number in dat_month.items():  
    
    month_patients_data_reading = pd.read_excel(rf'_{month}_patients_current_statuses_{today_y_m_d}.xlsx')
    month_patients_data = month_patients_data_reading.copy()

    
    month_patients_data['emiasid'] = month_patients_data['emiasid'].astype(str).str.replace('\.0', '')
    month_patients_data['date'] = month_patients_data['date'].apply(lambda x: pd.to_datetime(x).date())
    month_patients_data['next_call_date'] = month_patients_data['next_call_date'] \
    .apply(lambda x: pd.to_datetime(x).date())
    
    month_patients_medical_attachment_info = month_patients_data[['emiasid', 'doctor_fio', 'mo']]    
    month_patients_data = month_patients_data[['emiasid', 'date', 'status', 'old_comment', 'next_call_date']] 
    
    month_patients_calls_data = calls_data[calls_data.emiasid.isin(month_patients_data.emiasid)] \
    .reset_index(drop=True)


    # 'Исключить из задания'

    
    # 'Патронаж'
    month_patron_patients = month_patients_calls_data[
    month_patients_calls_data.talk_result == 'Патронажный пациент'] \
    .drop_duplicates(subset='emiasid')[['emiasid', 'call_date', 'talk_result', 'comment', 
    'next_call_date']].rename(columns={'call_date': 'date', 'talk_result': 'status', 
    'comment': 'old_comment'})
    month_patron_patients['status'] = month_patron_patients['status'] \
    .apply(lambda x: 'Патронаж')
    condition_for_month_patron = (month_patients_data.emiasid.isin(month_patron_patients.emiasid) &
    (~month_patients_data.status.isin({'Умер', 'Госпитализирован', 'Смена МО'})))
    month_patients_data = pd.concat([month_patients_data.drop(index=month_patients_data[
        condition_for_month_patron].index), month_patron_patients[month_patron_patients.emiasid.isin(
        month_patients_data[condition_for_month_patron].emiasid)]])
    month_patients_data = month_patients_data.reset_index(drop=True)
    
    

    # 'Некорректный номер'
    month_wrong_number_patients = month_patients_calls_data[month_patients_calls_data.call_result == 'Номер не существует'] \
    .drop_duplicates(subset='emiasid')[['emiasid', 'call_date', 'call_result', 'comment', 'next_call_date']] \
    .rename(columns={'call_date': 'date', 'call_result': 'status', 'comment': 'old_comment'})
    month_wrong_number_patients['status'] = 'Некорректный номер'
    condition_for_month_wrong_number = (month_patients_data.emiasid.isin(month_wrong_number_patients.emiasid) &
    (~month_patients_data.status.isin({'Умер', 'Госпитализирован', 'Смена МО', 'Патронаж'}) ))
    month_patients_data = pd.concat(
        [month_patients_data.drop(index=month_patients_data[condition_for_month_wrong_number].index), 
         month_wrong_number_patients[month_wrong_number_patients.emiasid.isin(month_patients_data[
        condition_for_month_wrong_number].emiasid)]])
    month_patients_data = month_patients_data.reset_index(drop=True)
  
        
        
    # 'Наблюдается в частной, федеральной или ведомственной мед организации'
    month_another_mo_patients = month_patients_calls_data[
        month_patients_calls_data.talk_result == 'Наблюдается в частной, федеральной или ведомственной мед организации'] \
    .drop_duplicates(subset='emiasid')[['emiasid', 'call_date', 'talk_result', 'comment', 'next_call_date']] \
    .rename(columns={'call_date': 'date', 'talk_result': 'status', 'comment': 'old_comment'})
    month_another_mo_patients['status'] = month_another_mo_patients['status'].apply(lambda x: 'Иная МО')
    condition_for_month_another_mo = (month_patients_data.emiasid.isin(month_another_mo_patients.emiasid)
    & (~month_patients_data.status.isin({'Умер', 'Госпитализирован', 'Смена МО', 'Патронаж', 'Некорректный номер'})))
    month_patients_data = pd.concat([month_patients_data.drop(index=month_patients_data[
    condition_for_month_another_mo].index), month_another_mo_patients[
    month_another_mo_patients.emiasid.isin(month_patients_data[
    condition_for_month_another_mo].emiasid)]])
    month_patients_data = month_patients_data.reset_index(drop=True)



    # 'Живет не в Москве'
    month_another_city_patients = month_patients_calls_data[
        month_patients_calls_data.talk_result == 'Живет не в Москве (России)'] \
    .drop_duplicates(subset='emiasid')[['emiasid', 'call_date', 'talk_result', 'comment', 'next_call_date']] \
    .rename(columns={'call_date': 'date', 'talk_result': 'status', 'comment': 'old_comment'})
    month_another_city_patients['status'] = month_another_city_patients['status'].apply(lambda x: 'Живет не в Москве')
    condition_for_month_another_city = (month_patients_data.emiasid.isin(month_another_city_patients.emiasid) &
    (~month_patients_data.status.isin({'Умер', 'Госпитализирован', 'Смена МО', 'Патронаж', 'Некорректный номер', 'Иная МО'})))
    month_patients_data = pd.concat([month_patients_data.drop(index=month_patients_data[
        condition_for_month_another_city].index), month_another_city_patients[
        month_another_city_patients.emiasid.isin(month_patients_data[condition_for_month_another_city] \
    .emiasid)]])
    month_patients_data = month_patients_data.reset_index(drop=True)


    # 'Будет в Москве в другом месяце'
    month_return_another_month_patients = month_patients_calls_data[
        month_patients_calls_data.talk_result == 'Будет в Москве в другом месяце'] \
    .drop_duplicates(subset='emiasid')[['emiasid', 'call_date', 'talk_result', 'comment', 'next_call_date']] \
    .rename(columns={'call_date': 'date', 'talk_result': 'status', 'comment': 'old_comment'})
    month_return_another_month_patients['status'] = month_return_another_month_patients['status'] \
    .apply(lambda x: 'Будет в Москве в другом месяце')
    condition_for_month_return_another_month = (month_patients_data.emiasid.isin(
        month_return_another_month_patients.emiasid) & (~month_patients_data.status.isin({
        'Умер', 'Госпитализирован', 'Смена МО', 'Патронаж', 'Некорректный номер', 'Иная МО', 'Живет не в Москве'})))
    month_patients_data = pd.concat([month_patients_data.drop(index=month_patients_data[
        condition_for_month_return_another_month].index), 
    month_return_another_month_patients[month_return_another_month_patients.emiasid.isin(
        month_patients_data[condition_for_month_return_another_month].emiasid)]])
    month_patients_data = month_patients_data.reset_index(drop=True)



    # 'Пациент отрицает диагноз'
    month_reject_diagnosis_patients = month_patients_calls_data[
        month_patients_calls_data.talk_result == 'Нет такого диагноза у пациента'] \
    .drop_duplicates(subset='emiasid')[['emiasid', 'call_date', 'talk_result', 'comment', 'next_call_date']] \
    .rename(columns={'call_date': 'date', 'talk_result': 'status', 'comment': 'old_comment'})
    month_reject_diagnosis_patients['status'] = month_reject_diagnosis_patients['status'] \
    .apply(lambda x: 'Пациент отрицает диагноз')
    condition_for_month_diagnosis_rejection = (month_patients_data.emiasid.isin(
        month_reject_diagnosis_patients.emiasid) & (~month_patients_data.status.isin({'Умер', 'Госпитализирован', 'Смена МО', 
    'Патронаж', 'Некорректный номер', 'Иная МО', 'Живет не в Москве', 'Будет в Москве в другом месяце'})))
    month_patients_data = pd.concat([month_patients_data.drop(index=month_patients_data[
        condition_for_month_diagnosis_rejection].index), 
    month_reject_diagnosis_patients[month_reject_diagnosis_patients.emiasid.isin(month_patients_data[
        condition_for_month_diagnosis_rejection].emiasid)]])
    month_patients_data = month_patients_data.reset_index(drop=True)


    # 'Cнят с ДН'
    month_not_in_dn_patients = month_patients_calls_data[
        month_patients_calls_data.talk_result == 'Диагноз ранее был, но с ДН снят'] \
    .drop_duplicates(subset='emiasid')[['emiasid', 'call_date', 'talk_result', 'comment', 'next_call_date']] \
    .rename(columns={'call_date': 'date', 'talk_result': 'status', 'comment': 'old_comment'})
    month_not_in_dn_patients['status'] = month_not_in_dn_patients['status'] \
    .apply(lambda x: 'Cнят с ДН')
    condition_for_month_not_in_dn = (month_patients_data.emiasid.isin(month_not_in_dn_patients.emiasid) &
    (~month_patients_data.status.isin({'Умер', 'Госпитализирован', 'Смена МО', 'Патронаж', 'Некорректный номер', 
    'Иная МО', 'Живет не в Москве', 'Будет в Москве в другом месяце', 'Пациент отрицает диагноз'})))
    month_patients_data = pd.concat([month_patients_data.drop(index=month_patients_data[
        condition_for_month_not_in_dn].index), 
    month_not_in_dn_patients[month_not_in_dn_patients.emiasid.isin(month_patients_data[
        condition_for_month_not_in_dn].emiasid)]])
    month_patients_data = month_patients_data.reset_index(drop=True)

    
    
        
    #  На третий цикл
    
    
 
    # 'Запишется самостоятельно' 
    month_patients_calls_data_2 = month_patients_calls_data.copy()
    month_patients_calls_data_2['talk_result'] = month_patients_calls_data_2['comment'].apply(lambda x: 
    'Запишется самостоятельно' if 'запишется' in str(x).lower() else x) 
    month_patients_calls_data_2.loc[month_patients_calls_data_2.talk_result == 'Запишется самостоятельно',
    'call_result'] = 'Запишется самостоятельно'
    month_self_record = month_patients_calls_data_2[month_patients_calls_data_2.call_result == \
    'Запишется самостоятельно'].drop_duplicates(subset='emiasid', keep='last')[['emiasid', 'call_date', 
    'call_result', 'comment', 'next_call_date']].rename(columns={'call_date': 'date', 
    'call_result': 'status', 'comment': 'old_comment'})
    condition_for_month_self_record = (month_patients_data.emiasid.isin(month_self_record.emiasid) &
    (~month_patients_data.status.isin({'Умер', 'Госпитализирован', 'Смена МО', 'Патронаж', 'Некорректный номер', 
    'Иная МО', 'Живет не в Москве', 'Будет в Москве в другом месяце', 'Пациент отрицает диагноз', 'Cнят с ДН'})))
    month_patients_data.loc[condition_for_month_self_record, 'status'] = 'Запишется самостоятельно'
    month_self_record_2 = month_patients_data[month_patients_data.status == 'Запишется самостоятельно'] \
    .sort_values(by='emiasid').reset_index(drop=True)[['emiasid', 'date', 'status']].merge(
    month_patients_calls_data_2[month_patients_calls_data_2.emiasid.isin(month_patients_data[
        condition_for_month_self_record].emiasid)].sort_values(by='call_date') \
        .drop_duplicates(subset='emiasid', keep='last').sort_values(by='emiasid') \
        .reset_index(drop=True)[['emiasid', 'call_date', 'call_result', 'comment', 'next_call_date']], 
        on='emiasid')
#     month_self_record_2.loc[((month_self_record_2.call_result.isin({'Разговор состоялся', 
#     'Запишется самостоятельно', 'Нет ответа', 'Не дозвонились', 'Занято', 'Сброс', 
#     'Перенос звонка на другой день'})) & 
#     month_self_record_2.next_call_date.isna()), 'next_call_date'] = month_self_record_2.loc[((
#     month_self_record_2.call_result.isin({'Разговор состоялся', 'Запишется самостоятельно', 'Нет ответа', 
#     'Не дозвонились', 'Занято', 'Сброс', 'Перенос звонка на другой день'})) & 
#     month_self_record_2.next_call_date.isna()), 'call_date'] + dt.timedelta(days=3)
    month_self_record_2.loc[(month_self_record_2.call_result.isin({'Перезвонить сегодня'})), 
    'next_call_date'] = month_self_record_2.loc[(month_self_record_2.call_result.isin({
        'Перезвонить сегодня'})), 'call_date']
    month_self_record_final_patients = month_self_record_2[['emiasid', 'date', 'status', 'comment', 
    'next_call_date']].rename(columns={'comment': 'old_comment'})
    month_patients_data = pd.concat([month_patients_data.drop(index=month_patients_data[
        condition_for_month_self_record].index), 
    month_self_record_final_patients[month_self_record_final_patients.emiasid.isin(month_patients_data[
        condition_for_month_self_record].emiasid)]]).sort_values('date').reset_index(drop=True)



    
    
    #  На следуюший цикл
    
    
    
    # 'Госпитализирован'
    month_hospitalized_patients = \
    month_patients_calls_data[month_patients_calls_data.call_result == 'Госпитализирован'] \
    .drop_duplicates(subset='emiasid')[['emiasid', 'call_date', 'call_result', 'comment', 'next_call_date']] \
    .rename(columns={'call_date': 'date', 'call_result': 'status', 'comment': 'old_comment'})
    
    condition_for_month_hospitalized_patients = (month_patients_data.emiasid.isin(
        month_hospitalized_patients.emiasid) & (~month_patients_data.status.isin({'Умер', 'Смена МО'})))
    month_patients_data.loc[condition_for_month_hospitalized_patients, 'status'] = 'Госпитализирован'
    month_patients_data = pd.concat(
        [month_patients_data.drop(index=month_patients_data[condition_for_month_hospitalized_patients].index), 
         month_hospitalized_patients[month_hospitalized_patients.emiasid.isin(month_patients_data[
        condition_for_month_hospitalized_patients].emiasid)]])
    month_patients_data = month_patients_data.reset_index(drop=True)
    
    
    
    # 'Отказ'
    month_rejected_patients = month_patients_calls_data[month_patients_calls_data.call_result == \
    'Отказ'].drop_duplicates(subset='emiasid')[['emiasid', 'call_date', 'call_result', 'comment', 
    'next_call_date']].rename(columns={'call_date': 'date', 'call_result': 'status', 
    'comment': 'old_comment'})
    condition_for_month_rejected_patients = (month_patients_data.emiasid.isin(month_rejected_patients.emiasid) &
    (~month_patients_data.status.isin({'Умер', 'Госпитализирован', 'Смена МО', 'Патронаж', 'Некорректный номер', 
    'Иная МО', 'Живет не в Москве', 'Будет в Москве в другом месяце', 'Пациент отрицает диагноз', 'Cнят с ДН',
    'Запишется самостоятельно'}) ))
    month_patients_data = pd.concat(
        [month_patients_data.drop(index=month_patients_data[condition_for_month_rejected_patients].index), 
         month_rejected_patients[month_rejected_patients.emiasid.isin(month_patients_data
        [condition_for_month_rejected_patients].emiasid)]])
    month_patients_data = month_patients_data.reset_index(drop=True)

    
    
    # 'Не нравится поликлиника и (или) все ее врачи'
    month_patients_calls_data_2 = month_patients_calls_data.copy()
    month_patients_calls_data_2.loc[month_patients_calls_data_2.talk_result == \
    'Не нравится поликлиника и (или) все ее врачи', 'call_result'] = 'Не нравится МО'
    month_mo_dislike = month_patients_calls_data_2[month_patients_calls_data_2.call_result == \
    'Не нравится МО'].drop_duplicates(subset='emiasid', keep='last')[['emiasid', 'call_date', 
    'call_result', 'comment', 'next_call_date']].rename(columns={'call_date': 'date', 
    'call_result': 'status', 'comment': 'old_comment'})
    condition_for_month_mo_dislike = (month_patients_data.emiasid.isin(month_mo_dislike.emiasid) &
    (~month_patients_data.status.isin({'Умер', 'Госпитализирован', 'Смена МО', 'Патронаж', 'Некорректный номер', 
    'Иная МО', 'Живет не в Москве', 'Будет в Москве в другом месяце', 'Пациент отрицает диагноз', 'Cнят с ДН',
    'Запишется самостоятельно', 'Отказ'})))
    month_patients_data.loc[condition_for_month_mo_dislike, 'status'] = 'Не нравится МО'
    month_mo_dislike_2 = month_patients_data[month_patients_data.status == 'Не нравится МО'] \
    .sort_values(by='emiasid').reset_index(drop=True)[['emiasid', 'date', 'status']] \
    .merge(month_patients_calls_data_2[month_patients_calls_data_2.emiasid.isin(
        month_patients_data[condition_for_month_mo_dislike].emiasid)] \
    .sort_values(by='call_date').drop_duplicates(subset='emiasid', keep='last') \
        .sort_values(by='emiasid').reset_index(drop=True)[['emiasid', 'call_date', 'call_result', 
        'comment', 'next_call_date']], on='emiasid')
    month_mo_dislike_2.loc[((month_mo_dislike_2.call_result.isin({'Разговор состоялся', 'Нет ответа', 
    'Не дозвонились', 'Занято', 'Сброс', 'Перенос звонка на другой день'})) & 
    month_mo_dislike_2.next_call_date.isna()), 'next_call_date'] = \
    month_mo_dislike_2.loc[((month_mo_dislike_2.call_result.isin({'Разговор состоялся', 'Нет ответа', 
    'Не дозвонились', 'Занято', 'Сброс', 'Перенос звонка на другой день'})) & 
    month_mo_dislike_2.next_call_date.isna()), 'call_date'] + dt.timedelta(days=7)
    month_mo_dislike_2.loc[(month_mo_dislike_2.call_result.isin({'Перезвонить сегодня'})), 
    'next_call_date'] = month_mo_dislike_2.loc[(month_mo_dislike_2.call_result.isin({
        'Перезвонить сегодня'})), 'call_date']
    month_mo_dislike_2_final_patients = month_mo_dislike_2[['emiasid', 'date', 'status', 'comment', 
    'next_call_date']].rename(columns={'comment': 'old_comment'})
    month_patients_data = pd.concat([month_patients_data.drop(index=month_patients_data[
        condition_for_month_mo_dislike].index), month_mo_dislike_2_final_patients[
        month_mo_dislike_2_final_patients.emiasid.isin(month_patients_data[
        condition_for_month_mo_dislike].emiasid)]]).sort_values('date').reset_index(drop=True)

    
    
    # 'Не нравится врач'
    month_patients_calls_data_2 = month_patients_calls_data.copy()
    month_patients_calls_data_2.loc[month_patients_calls_data_2.talk_result == \
    'Не хочет наблюдаться у данного участкового врача', 'call_result'] = 'Не нравится врач'
    month_doctor_dislike = month_patients_calls_data_2[month_patients_calls_data_2.call_result == \
    'Не нравится врач'].drop_duplicates(subset='emiasid', keep='last')[['emiasid', 'call_date', 
    'call_result', 'comment', 'next_call_date']].rename(columns={'call_date': 'date', 
    'call_result': 'status', 'comment': 'old_comment'})
    condition_for_month_doctor_dislike = (month_patients_data.emiasid.isin(month_doctor_dislike.emiasid) &
    (~month_patients_data.status.isin({'Умер', 'Госпитализирован', 'Смена МО', 'Патронаж', 'Некорректный номер', 
    'Иная МО', 'Живет не в Москве', 'Будет в Москве в другом месяце', 'Пациент отрицает диагноз', 'Cнят с ДН',
    'Запишется самостоятельно', 'Отказ', 'Не нравится МО'})))
    month_patients_data.loc[condition_for_month_doctor_dislike, 'status'] = 'Не нравится врач'
    month_doctor_dislike_2 = month_patients_data[month_patients_data.status == 'Не нравится врач'] \
    .sort_values(by='emiasid').reset_index(drop=True)[['emiasid', 'date', 'status']].merge(
    month_patients_calls_data_2[month_patients_calls_data_2.emiasid.isin(month_patients_data[
    condition_for_month_doctor_dislike].emiasid)].sort_values(by='call_date') \
        .drop_duplicates(subset='emiasid', keep='last').sort_values(by='emiasid') \
    .reset_index(drop=True)[['emiasid', 'call_date', 'call_result', 'comment', 'next_call_date']], 
        on='emiasid')
    month_doctor_dislike_2.loc[((month_doctor_dislike_2.call_result.isin({'Разговор состоялся', 
    'Нет ответа', 'Не дозвонились', 'Занято', 'Сброс', 'Перенос звонка на другой день'})) & 
    month_doctor_dislike_2.next_call_date.isna()), 'next_call_date'] = month_doctor_dislike_2.loc[((
        month_doctor_dislike_2.call_result.isin({'Разговор состоялся', 'Нет ответа', 
    'Не дозвонились', 'Занято', 'Сброс', 'Перенос звонка на другой день'})) & 
        month_doctor_dislike_2.next_call_date.isna()), 'call_date'] + dt.timedelta(days=7)
    month_doctor_dislike_2.loc[(month_doctor_dislike_2.call_result.isin({'Перезвонить сегодня'})), 
    'next_call_date'] = month_doctor_dislike_2.loc[(month_doctor_dislike_2.call_result.isin({
        'Перезвонить сегодня'})), 'call_date']
    month_doctor_dislike_2_final_patients = month_doctor_dislike_2[['emiasid', 'date', 'status', 
    'comment', 'next_call_date']].rename(columns={'comment': 'old_comment'})
    month_patients_data = pd.concat([month_patients_data.drop(index=month_patients_data[
    condition_for_month_doctor_dislike].index), month_doctor_dislike_2_final_patients[
        month_doctor_dislike_2_final_patients.emiasid.isin(month_patients_data[
    condition_for_month_doctor_dislike].emiasid)]]).sort_values('date').reset_index(drop=True)

    # 'Другой врач в этой МО'
    month_patients_calls_data_2 = month_patients_calls_data.copy()
    month_patients_calls_data_2.loc[month_patients_calls_data_2.talk_result == \
    'Лечащий врач другой специалист в этой поликлинике', 'call_result'] = 'Другой врач в этой МО'
    month_another_doctor = month_patients_calls_data_2[month_patients_calls_data_2.call_result == \
    'Другой врач в этой МО'].drop_duplicates(subset='emiasid', keep='last')[['emiasid', 'call_date', 
    'call_result', 'comment', 'next_call_date']].rename(columns={'call_date': 'date', 
    'call_result': 'status', 'comment': 'old_comment'})
    condition_for_month_another_doctor = (month_patients_data.emiasid.isin(month_another_doctor.emiasid) &
    (~month_patients_data.status.isin({'Умер', 'Госпитализирован', 'Смена МО', 'Патронаж', 'Некорректный номер', 
    'Иная МО', 'Живет не в Москве', 'Будет в Москве в другом месяце', 'Пациент отрицает диагноз', 'Cнят с ДН',
    'Запишется самостоятельно', 'Отказ', 'Не нравится МО'})))
    month_patients_data.loc[condition_for_month_another_doctor, 'status'] = 'Другой врач в этой МО'
    month_another_doctor_2 = month_patients_data[month_patients_data.status == 'Другой врач в этой МО'] \
    .sort_values(by='emiasid').reset_index(drop=True)[['emiasid', 'date', 'status']].merge(
    month_patients_calls_data_2[month_patients_calls_data_2.emiasid.isin(month_patients_data[
    condition_for_month_another_doctor].emiasid)].sort_values(by='call_date').drop_duplicates(subset= \
    'emiasid', keep='last').sort_values(by='emiasid').reset_index(drop=True)[['emiasid', 'call_date', 
    'call_result', 'comment', 'next_call_date']], on='emiasid')
    month_another_doctor_2.loc[((month_doctor_dislike_2.call_result.isin({'Разговор состоялся', 
    'Нет ответа', 'Не дозвонились', 'Занято', 'Сброс', 'Перенос звонка на другой день'})) & 
    month_another_doctor_2.next_call_date.isna()), 'next_call_date'] = \
    month_another_doctor_2.loc[((month_doctor_dislike_2.call_result.isin({'Разговор состоялся', 
    'Нет ответа', 'Не дозвонились', 'Занято', 'Сброс', 'Перенос звонка на другой день'})) & 
    month_another_doctor_2.next_call_date.isna()), 'call_date'] + dt.timedelta(days=7)
    month_another_doctor_2.loc[(month_another_doctor_2.call_result.isin({'Перезвонить сегодня'})), 
    'next_call_date'] = month_another_doctor_2.loc[(month_another_doctor_2.call_result.isin({
        'Перезвонить сегодня'})), 'call_date']
    month_another_doctor_2_final_patients = month_another_doctor_2[['emiasid', 'date', 'status', 
    'comment', 'next_call_date']].rename(columns={'comment': 'old_comment'})
    month_patients_data = pd.concat([month_patients_data.drop(index=month_patients_data[
        condition_for_month_another_doctor].index), month_another_doctor_2_final_patients[
        month_another_doctor_2_final_patients.emiasid.isin(month_patients_data[
        condition_for_month_another_doctor].emiasid)]]).sort_values('date').reset_index(drop=True)

    # 'Перенос звонка на другой день', 'Не дозвонились', 'Перезвонить сегодня'
    
    month_patients_calls_data_2 = month_patients_calls_data.copy()
    month_patients_calls_data_2 = month_patients_calls_data_2[~month_patients_calls_data_2.emiasid.isin(
    month_patients_data[month_patients_data.status.isin({'Умер', 'Госпитализирован', 'Смена МО', 'Патронаж', 
    'Некорректный номер', 'Иная МО', 'Живет не в Москве', 'Будет в Москве в другом месяце', 'Пациент отрицает диагноз', 
    'Cнят с ДН', 'Запишется самостоятельно', 'Отказ', 'Не нравится МО', 'Другой врач в этой МО', 'Не нравится врач'})].emiasid)]

    month_patients_calls_data_2 = month_patients_calls_data_2.sort_values(by='call_date') \
    .drop_duplicates(subset='emiasid', keep='last')
    month_patients_calls_data_2.loc[month_patients_calls_data_2.call_result.isin({'Не дозвонились', 
    'Нет ответа', 'Занято', 'Сброс'}), 'call_result'] = 'Не дозвонились'
    month_misscall_patients = month_patients_calls_data_2[month_patients_calls_data_2.call_result == \
    'Не дозвонились']
    month_misscall_patients = month_misscall_patients[['emiasid', 'call_date', 'call_result', 'comment', 
    'next_call_date']].rename(columns={'call_date': 'date', 'call_result': 'status', 
    'comment': 'old_comment'})
    
    month_another_day = month_patients_calls_data_2[month_patients_calls_data_2.call_result == \
                                                    'Перенос звонка на другой день']
    month_another_day = month_another_day[['emiasid', 'call_date', 'call_result', 'comment', 
    'next_call_date']].rename(columns={'call_date': 'date', 'call_result': 'status', 
    'comment': 'old_comment'})
    
#     month_another_day.loc[month_another_day.next_call_date.isna(), 'next_call_date'] = \
#     month_another_day.loc[month_another_day.next_call_date.isna(), 'date'] + dt.timedelta(days=3)
    
    month_same_day = month_patients_calls_data_2[month_patients_calls_data_2.call_result == \
                                                 'Перезвонить сегодня']
    month_same_day = month_same_day[['emiasid', 'call_date', 'call_result', 'comment', 
    'next_call_date']].rename(columns={'call_date': 'date', 'call_result': 'status', 
    'comment': 'old_comment'})
    month_same_day.loc[month_same_day.next_call_date.isna(), 'next_call_date'] = \
    month_same_day.loc[month_same_day.next_call_date.isna(), 'date']
    
    
    month_patients_data = pd.concat([month_patients_data.drop(index=month_patients_data[
        month_patients_data.emiasid.isin(month_misscall_patients.emiasid)].index), month_misscall_patients[
        month_misscall_patients.emiasid.isin(month_patients_data[month_patients_data.emiasid.isin(
            month_misscall_patients.emiasid)].emiasid)]]).sort_values('date').reset_index(drop=True)
    month_patients_data = pd.concat([month_patients_data.drop(index=month_patients_data[
        month_patients_data.emiasid.isin(month_another_day.emiasid)].index), month_another_day[
        month_another_day.emiasid.isin(month_patients_data[month_patients_data.emiasid.isin(
            month_another_day.emiasid)].emiasid)]]).sort_values('date').reset_index(drop=True)
    month_patients_data = pd.concat([month_patients_data.drop(index=month_patients_data[
        month_patients_data.emiasid.isin(month_same_day.emiasid)].index), month_same_day[
        month_same_day.emiasid.isin(month_patients_data[month_patients_data.emiasid.isin(
        month_same_day.emiasid)].emiasid)]]).sort_values('date').reset_index(drop=True)

    # 'Разговор состоялся'
    talken_patients = month_patients_calls_data_2[(month_patients_calls_data_2.call_result == 'Разговор состоялся') & 
    (month_patients_calls_data_2.talk_result.isna()) & (~month_patients_calls_data_2.emiasid.isin(
    month_patients_data[month_patients_data.status.isin({'Умер', 'Госпитализирован', 'Смена МО', 'Патронаж', 
    'Некорректный номер', 'Иная МО', 'Живет не в Москве', 'Будет в Москве в другом месяце', 'Пациент отрицает диагноз', 
    'Cнят с ДН', 'Запишется самостоятельно', 'Отказ', 'Не нравится МО', 'Другой врач в этой МО', 'Не нравится врач',
    'Не дозвонились', 'Перенос звонка на другой день', 'Перезвонить сегодня'})].emiasid))].drop_duplicates(subset='emiasid', 
    keep='last').reset_index(drop=True)
    talken_patients = talken_patients[['emiasid', 'call_date', 'call_result', 'comment', 'next_call_date']].rename(
    columns={'call_date':'date', 'call_result': 'status', 'comment': 'old_comment'})
    month_patients_data = pd.concat([month_patients_data.drop(index=month_patients_data[
        month_patients_data.emiasid.isin(talken_patients.emiasid)].index), talken_patients[
        talken_patients.emiasid.isin(month_patients_data[month_patients_data.emiasid.isin(
        talken_patients.emiasid)].emiasid)]]).sort_values('date').reset_index(drop=True)

#     # 'Ни разу не дозвонились'
#     month_misscall_patients_calls = month_patients_calls_data[month_patients_calls_data.emiasid.isin(
#         month_misscall_patients.emiasid)]
#     month_at_least_one_succesful_call_patients = set(month_misscall_patients_calls[month_misscall_patients_calls \
#         .call_result.isin({
#         'Перезвонить сегодня', 'Записан на приём',
#         'Перенос звонка на другой день', 'Разговор состоялся'
#     })].emiasid)
#     totally_misscall_patients = month_misscall_patients[~month_misscall_patients.emiasid \
#     .isin(month_at_least_one_succesful_call_patients)]
#     totally_misscall_patients = month_misscall_patients_calls[month_misscall_patients_calls.emiasid.isin(
#         totally_misscall_patients.emiasid)].drop_duplicates(subset='emiasid', keep='last') \
#     .reset_index(drop=True)[['emiasid', 'call_date', 'call_result', 'comment', 'next_call_date']].rename(
#     columns={'call_date':'date', 'call_result': 'status', 'comment': 'old_comment'})
#     totally_misscall_patients['status'] = 'Ни разу не дозвонились'
#     month_patients_data = pd.concat([month_patients_data.drop(index=month_patients_data[
#     month_patients_data.emiasid.isin(totally_misscall_patients.emiasid)].index), totally_misscall_patients[
#     totally_misscall_patients.emiasid.isin(month_patients_data[month_patients_data.emiasid.isin(
#     totally_misscall_patients.emiasid)].emiasid)]]).sort_values('date').reset_index(drop=True)

    # 'Ни разу не звонили'
    month_patients_data.loc[(~month_patients_data.emiasid.isin(month_patients_calls_data.emiasid)) & 
                            (~month_patients_data.status.isin({'Умер', 'Госпитализирован', 'Смена МО', 'Некорректный номер'})),
                            'status'] = 'Ни разу не звонили'

    # 'без статуса ==>> Разговор состоялся'
    no_status_patients = month_patients_data[month_patients_data.status == 'без статуса'] \
    .reset_index(drop=True)
    no_status_patients_calls = month_patients_calls_data[month_patients_calls_data.emiasid.isin(
        no_status_patients.emiasid)].drop_duplicates(subset='emiasid', keep='last').reset_index(drop=True)
    no_status_patients = no_status_patients_calls[['emiasid', 'call_date', 'call_result', 'comment', 
    'next_call_date']].rename(columns={'call_date':'date', 'call_result': 'status', 
    'comment': 'old_comment'})
    no_status_patients['status'] = 'Разговор состоялся'
    month_patients_data = pd.concat([month_patients_data.drop(index=month_patients_data[
    month_patients_data.emiasid.isin(no_status_patients.emiasid)].index), no_status_patients[
    no_status_patients.emiasid.isin(month_patients_data[month_patients_data.emiasid.isin(
    no_status_patients.emiasid)].emiasid)]]).sort_values('date').reset_index(drop=True)

    
    

    # 'завершение'
    month_patients_data = month_patients_data.merge(month_patients_medical_attachment_info, on='emiasid')
    month_patients_data['date'] = month_patients_data['date'].apply(lambda x: pd.to_datetime(x).date())
    month_patients_data['next_call_date'] = month_patients_data['next_call_date'] \
    .apply(lambda x: pd.to_datetime(x).date())
    
    month_patients_data.merge(assistant_to_doctor_distribution[['mo', 'doctor_fio', 'login']], 
                              on=['doctor_fio', 'mo'], how='left') \
    .sort_values(by='date').to_excel(rf'{month}_patients_current_statuses_{today_y_m_d}.xlsx', index=False)

<timed exec>:11: FutureWarning: The default value of regex will change from True to False in a future version.
<timed exec>:23: FutureWarning: The default value of regex will change from True to False in a future version.


CPU times: total: 40.1 s
Wall time: 40.3 s


In [ ]:
## 54 s

In [ ]:
month_patients_calls_data.emiasid.nunique()

# Уже после формирования всех статусов после анализа емиаса и звонков приходит первичное распределение на основе раскрепления 
# из файла исправление задания для августа

In [ ]:
'без статуса' - 'на первый цикл',
'Ни разу не звонили' - 'на первый цикл', 



'Некорректный номер' - 'исключить из задания',                                
'Пациент отрицает диагноз' - 'исключить из задания', 
'Живет не в Москве' - 'исключить из задания',
'Будет в Москве в другом месяце' - 'исключить из задания', 
'Иная МО' - 'исключить из задания', 
'Cнят с ДН' - 'исключить из задания',
'Патронаж' - 'исключить из задания', 
'Умер' - 'исключить из задания'
Два раза 'Отказ' - 'Два отказа' -'исключить из задания'
Два раза 'Не дозвонились' - 'Ни разу не дозвонились'  - 'исключить из задания'



'Перезвонить сегодня' - 'на следуюший день', 



'Перенос звонка на другой день' - 'на определенную дату', 



'Не дозвонились' - 'на следующий цикл',
'Отказ' - 'на следующий цикл',
'Госпитализирован' - 'на следующий цикл', 
'Другой врач в этой МО' - 'на следующий цикл',
'Не нравится МО' - 'на следующий цикл', 
'Не нравится врач' - 'на следующий цикл',


'Разговор состоялся'  - 'на третий цикл', 
'Запишется самостоятельно'  - 'на третий цикл', 


'Все прошли' - 'Через неделю'